# Project_Data

### Summary
1. Claim and Related Article Data:
    - Made lowercase, removed punctuations, links, unicode hex amongst other misc items like ", ', - ...etc.
    - Removed stopwords and Tokenized
        - To run this section, you may have to download the stopwords packages. I have included the code, you just have to uncomment 2 lines on the first run (section 1.3)
    - *new - November 15* implemented sentence extraction and applied the same processing as above
2. Date
    - Converted from string to datetime format (for practicality)
    - Created 3 features:
        - 1. Days since Jan 1st 1986
        - 2. The Month
        - 3. The Year
3. Claimant
    - Replaced missing values with "unknown"
    - Replaced counts below threshold with "other"
4. Final Frame
    - 2 final frames:
        - final_data = this is the frame that holds the claims, claimant, date, label, related articles
        - final_articles = this is the frame that holds the related articles
    - I have included a few extra lines of code as an example of how to work with the frames
        - I have saved the output of the 2 dataframe to csv files

## Preliminaries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from datetime import datetime
import os
import math
from IPython.display import clear_output, display
import time
import warnings
warnings.filterwarnings('ignore')
import string

# The following line is needed to show plots inline in notebooks
%matplotlib inline 

In [2]:
# function to convert strings to numpy array - used to convert the related_articles column in to arrays
# for practicality
def str2array(value):
    str_list = re.findall(r'\d+', value)
    int_list = list(map(int, str_list))
    article_array = np.array(int_list)
    return article_array

In [3]:
data = pd.read_csv('train.csv')
data.shape

(15555, 7)

In [4]:
# create a new column with the related articles saved as an array called "article_array"
data['article_array'] = data['related_articles'].apply(str2array)
data.head()

,Unnamed: 0,claim,claimant,date,id,label,related_articles,article_array
0,0,A line from George Orwell's novel 1984 predict...,NaN,17/07/2017,0,0,"[122094, 122580, 130685, 134765]","[122094, 122580, 130685, 134765]"
1,1,Maine legislature candidate Leslie Gibson insu...,NaN,17/03/2018,1,2,"[106868, 127320, 128060]","[106868, 127320, 128060]"
2,2,A 17-year-old girl named Alyssa Carson is bein...,NaN,18/07/2018,4,1,"[132130, 132132, 149722]","[132130, 132132, 149722]"
3,3,In 1988 author Roald Dahl penned an open lette...,NaN,04/02/2019,5,2,"[123254, 123418, 127464]","[123254, 123418, 127464]"
4,4,"When it comes to fighting terrorism, ""Another ...",Hillary Clinton,22/03/2016,6,2,"[41099, 89899, 72543, 82644, 95344, 88361]","[41099, 89899, 72543, 82644, 95344, 88361]"


In [5]:
# set paths
cur_path = os.path.dirname(os.path.abspath("Project_Data.ipynb"))
articles_dir = cur_path + '/train_articles/'

In [6]:
%%time
# create a dictionary of article ID and content
article_dict = {}
for filename in os.listdir(articles_dir):
    filenumber = filename.replace('.txt', '')
    file_open = open(articles_dir + filename, "r")
    text = file_open.read()
    article_dict[filenumber] = text
# use the dictionary created to create a dataframe of articles
articles  = pd.DataFrame.from_dict(article_dict, orient='index')
articles.columns = ['Article']
# a dataframe that holds all the articles
articles.head()

CPU times: user 1.39 s, sys: 466 ms, total: 1.85 s
Wall time: 1.86 s


,Article
125385,Dr. Ben Carson: Welfare Benefactor?\nAn initia...
32238,The World Factbook — Central Intelligence Agen...
16051,"1014 texaseducationagencypftexas\n\nEmails, La..."
118633,Clinton camp delays Weather Channel ad buy aft...
117945,Living with kangaroos\nKangaroos are appealing...


## 1. Related Articles and Claim Data

### 1.1 Basic Cleaning for Related Articles

In [7]:
%%time
# CLEAN ARTICLE DATA - ~5 minutes to run
# convert all string values to lower case
articles_cleaned = articles.apply(lambda x: x.str.lower())
# replace new line with space
articles_cleaned = articles_cleaned.replace('\n', ' ', regex=True)
# get rid of all links
articles_cleaned = articles_cleaned.Article.replace(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}     /)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', regex = True).to_frame()
# get rid of unicode hex
articles_cleaned = articles_cleaned.Article.replace({r'[^\x00-\x7F]+':''}, regex=True).to_frame()
# remove punctuation
# articles_cleaned = articles_cleaned.Article.str.replace('[{}]'.format(string.punctuation), '').to_frame()
# remove misc items
articles_cleaned = articles_cleaned.replace(' — ', ' ', regex=True)
articles_cleaned = articles_cleaned.replace('-', '', regex=True)
articles_cleaned = articles_cleaned.replace('’', '', regex=True)
articles_cleaned = articles_cleaned.replace('‘', '', regex=True)
articles_cleaned = articles_cleaned.replace('”', '', regex=True)
articles_cleaned = articles_cleaned.replace('“', '', regex=True)
# replace consecutive spaces with just one space
articles_cleaned = articles_cleaned.replace('\s+', ' ', regex=True)

CPU times: user 1min 53s, sys: 454 ms, total: 1min 53s
Wall time: 1min 53s


#### Extract Sentences from Related Articles

In [8]:
%%time

# split each article in to sentences
from nltk import sent_tokenize
article_sentences = []
for i in range(articles_cleaned.shape[0]):
    sentence = articles_cleaned.Article[articles_cleaned.index[i]]
    tok_sen = sent_tokenize(sentence)
    article_sentences.append(tok_sen)

CPU times: user 2min, sys: 620 ms, total: 2min
Wall time: 2min


In [9]:
# remove punctuation from cleaned articles - had to do this after the sentences are separated
articles_cleaned = articles_cleaned.Article.str.replace('[{}]'.format(string.punctuation), '').to_frame()

In [10]:
%%time

# remove punctuation from sentences
cleaned_article_sent = []
for i in range(articles_cleaned.shape[0]):
    cleaned_sent = [''.join(c for c in s if c not in string.punctuation) for s in article_sentences[i]]
    # cleaned_article_sent is a list of lists
    # the sentences from each article is grouped in to a list, and then grouped together based on article
    cleaned_article_sent.append(cleaned_sent)

CPU times: user 48.6 s, sys: 296 ms, total: 48.8 s
Wall time: 48.9 s


In [11]:
# convert the cleaned_Article_sent in to a dataframe that's indexed by article ID
article_sentences = pd.Series(cleaned_article_sent).to_frame()
article_sentences.columns = ['Sentence']
article_sentences.index = articles_cleaned.index

In [12]:
# article sentences in a dataframe
article_sentences.head()

,Sentence
125385,"[dr ben carson welfare benefactor, an initiall..."
32238,[the world factbook central intelligence agenc...
16051,[1014 texaseducationagencypftexas emails laure...
118633,[clinton camp delays weather channel ad buy af...
117945,[living with kangaroos kangaroos are appealing...


### 1.2 Basic Cleaning for Claims 

In [16]:
%%time
# CLEAN CLAIM DATA
# create a new dataframe of just claims
cleaned_claim = data.claim.to_frame()
# convert all string values to lower case
cleaned_claim = cleaned_claim.apply(lambda x: x.str.lower())
# replace new line with space
cleaned_claim = cleaned_claim.replace('\n', ' ', regex=True)
# get rid of all links
cleaned_claim = cleaned_claim.claim.replace(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}     /)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', regex = True).to_frame()
# get rid of unicode hex
cleaned_claim = cleaned_claim.claim.replace({r'[^\x00-\x7F]+':''}, regex=True).to_frame()
# remove punctuation
cleaned_claim = cleaned_claim.claim.str.replace('[{}]'.format(string.punctuation), '').to_frame()
# remove misc items
cleaned_claim = cleaned_claim.replace(' — ', ' ', regex=True)
cleaned_claim = cleaned_claim.replace('-', ' ', regex=True)
cleaned_claim = cleaned_claim.replace('’', '', regex=True)
cleaned_claim = cleaned_claim.replace('‘', '', regex=True)
cleaned_claim = cleaned_claim.replace('”', '', regex=True)
cleaned_claim = cleaned_claim.replace('“', '', regex=True)
# replace consecutive spaces with just one space
cleaned_claim = cleaned_claim.replace('\s+', ' ', regex=True)

CPU times: user 479 ms, sys: 0 ns, total: 479 ms
Wall time: 477 ms


In [17]:
# concatenate cleaned_claims with label and article_array
cleaned_claim = pd.concat([cleaned_claim, data.label, data.article_array], axis=1)
# cleaned_claim now holds the claims that are cleaned, the label, and the article array
cleaned_claim.head()

,claim,label,article_array
0,a line from george orwells novel 1984 predicts...,0,"[122094, 122580, 130685, 134765]"
1,maine legislature candidate leslie gibson insu...,2,"[106868, 127320, 128060]"
2,a 17yearold girl named alyssa carson is being ...,1,"[132130, 132132, 149722]"
3,in 1988 author roald dahl penned an open lette...,2,"[123254, 123418, 127464]"
4,when it comes to fighting terrorism another th...,2,"[41099, 89899, 72543, 82644, 95344, 88361]"


### 1.3 Stemming, Stop Words and Tokenization 

In [18]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
# the first time running - you may need to uncomment the bottom two lines to download the necessary packages
# nltk.download('stopwords')
# nltk.download('punkt')

### 1.3.1 Claims

In [19]:
# create a list of claims
claim_list=[]
for i in range(cleaned_claim.shape[0]):
    claim_entry = cleaned_claim.claim.loc[i]
    claim_list.append(claim_entry)

In [20]:
%%time
# tokenize every claim in the claim list generated from above
# the result is a list of tokenized claims: tokenized_claims
tokenized_claims = []
stemmed_claims = []
stemmed_sw_claims = []
for i in range(cleaned_claim.shape[0]):

    #--------------------------------------------------------------
    # stemming
    word_tokens = word_tokenize(claim_list[i])
    stemmed_tok_claims = []
    for w in word_tokens:
        stemmed_tok_claims.append(ps.stem(w))
    stemmed_string = ' '.join(stemmed_tok_claims)
    # stemmed_claims is a list of stemmed strings
    stemmed_claims.append(stemmed_string)
    
    #--------------------------------------------------------------
    # remove stop words
    stemmed_sw_string = []
    word_tokens = word_tokenize(stemmed_claims[i])
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    stemmed_sw_string = ' '.join(filtered_sentence)
    # stemmed_sw_claims is a list of stemmed strings without stopwords
    stemmed_sw_claims.append(stemmed_sw_string)    
        
    #--------------------------------------------------------------    
    # tokenize
    tokenized_ = word_tokenize(stemmed_sw_claims[i])
    tokenized_claims.append(tokenized_)
    
    # print progress
    progress = round((i/cleaned_claim.shape[0])*100,2)
    clear_output(wait=True)
    print("progress: " + str(progress) + "%")

progress: 99.99%
CPU times: user 33.6 s, sys: 2.39 s, total: 36 s
Wall time: 33 s


In [ ]:
# list of tokenized claims
# tokenized_claims

#### Create Claims Dataframe

In [21]:
# zip together all the claim lists and create a dataframe
zipped_claims = list(zip(stemmed_claims, stemmed_sw_claims, tokenized_claims))
claims_ = pd.DataFrame(zipped_claims, columns = ['stemmed_claims', 'stemmed_stopword_claims', 'tokenized_claims'])

In [22]:
claims_.head()

,stemmed_claims,stemmed_stopword_claims,tokenized_claims
0,a line from georg orwel novel 1984 predict the...,line georg orwel novel 1984 predict power smar...,"[line, georg, orwel, novel, 1984, predict, pow..."
1,main legislatur candid lesli gibson insult par...,main legislatur candid lesli gibson insult par...,"[main, legislatur, candid, lesli, gibson, insu..."
2,a 17yearold girl name alyssa carson is be trai...,17yearold girl name alyssa carson train nasa b...,"[17yearold, girl, name, alyssa, carson, train,..."
3,in 1988 author roald dahl pen an open letter u...,1988 author roald dahl pen open letter urg par...,"[1988, author, roald, dahl, pen, open, letter,..."
4,when it come to fight terror anoth thing we kn...,come fight terror anoth thing know doe work ba...,"[come, fight, terror, anoth, thing, know, doe,..."


### 1.3.2 Related Articles

In [23]:
%%time
# create a list of tokenized, non-stop words articles ~ takes 30 min
tokenized_articles = []
stemmed_art = []
stemmed_sw_art = []

for i in range(articles_cleaned.shape[0]):

    #--------------------------------------------------------------
    # stemming
    word_tokens = word_tokenize(articles_cleaned.Article[articles_cleaned.index[i]])
    stemmed_tok_art = []
    for w in word_tokens:
        stemmed_tok_art.append(ps.stem(w))
    stemmed_string = ' '.join(stemmed_tok_art)
    # stemmed_claims is a list of stemmed strings
    stemmed_art.append(stemmed_string)
    
    #--------------------------------------------------------------
    # remove stop words
    stemmed_sw_string = []
    word_tokens = word_tokenize(stemmed_art[i])
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    stemmed_sw_string = ' '.join(filtered_sentence)
    # stemmed_sw_claims is a list of stemmed strings without stopwords
    stemmed_sw_art.append(stemmed_sw_string)    
    
    #--------------------------------------------------------------    
    # tokenize
    tokenized_ = word_tokenize(stemmed_sw_art[i])
    tokenized_articles.append(tokenized_)
    
    # print progress
    progress = round((i/articles_cleaned.shape[0])*100,2)
    clear_output(wait=True)
    print("progress: " + str(progress) + "%")

progress: 100.0%
CPU times: user 31min 18s, sys: 20 s, total: 31min 38s
Wall time: 31min 22s


#### Sentences for Related Articles

In [87]:
%%time

# stem, remove stop words from sentences ~ takes 30 min

one_article = []
final_article_sentences = []
tokenized_sentences = []
final_article_tok_sentences = []

for i in range(article_sentences.shape[0]):
    # grab a series of sentence for one article
    sentence_list = article_sentences.Sentence[article_sentences.index[i]]
    for u in range(len(sentence_list)):
        # stem, stopwords for each sentence in an article
        sent = sentence_list[u]
        word_tokens = word_tokenize(sent)
        stemmed_tok_sent = []
        for w in word_tokens:
            stemmed_tok_sent.append(ps.stem(w))
        filtered_sentence = [w for w in stemmed_tok_sent if not w in stop_words]
        # append the tokenized strings
        tokenized_sentences.append(filtered_sentence)
        # append the strings
        stemmed_sw_string = ' '.join(filtered_sentence)
        one_article.append(stemmed_sw_string)
    # append each one_article to a full list of all articles
    final_article_sentences.append(one_article)  
    final_article_tok_sentences.append(tokenized_sentences)
    one_article = []
    tokenized_sentences = []
    
    # print progress
    progress = round((i/article_sentences.shape[0])*100,2)
    clear_output(wait=True)
    print("progress: " + str(progress) + "%")    

progress: 100.0%
CPU times: user 29min 28s, sys: 27.2 s, total: 29min 55s
Wall time: 29min 39s


In [88]:
# article_sentences is the dataframe that holds the sentences
article_sentences['cleaned_sentence'] = final_article_sentences
article_sentences['tokenized_cleaned_sentence'] = final_article_tok_sentences
article_sentences.head()

,Sentence,cleaned_sentence,tokenized_cleaned_sentence
125385,"[dr ben carson welfare benefactor, an initiall...","[dr ben carson welfar benefactor, initi unlik ...","[[dr, ben, carson, welfar, benefactor], [initi..."
32238,[the world factbook central intelligence agenc...,[world factbook central intellig agenc unit st...,"[[world, factbook, central, intellig, agenc, u..."
16051,[1014 texaseducationagencypftexas emails laure...,[1014 texaseducationagencypftexa email lauren ...,"[[1014, texaseducationagencypftexa, email, lau..."
118633,[clinton camp delays weather channel ad buy af...,[clinton camp delay weather channel ad buy bac...,"[[clinton, camp, delay, weather, channel, ad, ..."
117945,[living with kangaroos kangaroos are appealing...,[live kangaroo kangaroo appeal wild power nati...,"[[live, kangaroo, kangaroo, appeal, wild, powe..."


In [ ]:
# list of tokenized related articles - below is showing only the first entry of the list
# tokenized_articles[0]

#### Create Related Articles DataFrame

In [92]:
# zip together all the articles and create a dataframe
zipped_articles = list(zip(stemmed_art, stemmed_sw_art, tokenized_articles, final_article_sentences, final_article_tok_sentences))
articles_ = pd.DataFrame(zipped_articles, columns = ['stemmed_articles', 'stemmed_stopword_articles', 'tokenized_articles', 'cleaned_sentence', 'tokenized_cleaned_sentence'])
# index the articles based on article ID
articles_.index = [articles_cleaned.index]

In [93]:
articles_.head()

,stemmed_articles,stemmed_stopword_articles,tokenized_articles,cleaned_sentence,tokenized_cleaned_sentence
125385,dr ben carson welfar benefactor an initi unlik...,dr ben carson welfar benefactor initi unlik cl...,"[dr, ben, carson, welfar, benefactor, initi, u...","[dr ben carson welfar benefactor, initi unlik ...","[[dr, ben, carson, welfar, benefactor], [initi..."
32238,the world factbook central intellig agenc the ...,world factbook central intellig agenc unit sta...,"[world, factbook, central, intellig, agenc, un...",[world factbook central intellig agenc unit st...,"[[world, factbook, central, intellig, agenc, u..."
16051,1014 texaseducationagencypftexa email lauren c...,1014 texaseducationagencypftexa email lauren c...,"[1014, texaseducationagencypftexa, email, laur...",[1014 texaseducationagencypftexa email lauren ...,"[[1014, texaseducationagencypftexa, email, lau..."
118633,clinton camp delay weather channel ad buy afte...,clinton camp delay weather channel ad buy back...,"[clinton, camp, delay, weather, channel, ad, b...",[clinton camp delay weather channel ad buy bac...,"[[clinton, camp, delay, weather, channel, ad, ..."
117945,live with kangaroo kangaroo are appeal wild an...,live kangaroo kangaroo appeal wild power nativ...,"[live, kangaroo, kangaroo, appeal, wild, power...",[live kangaroo kangaroo appeal wild power nati...,"[[live, kangaroo, kangaroo, appeal, wild, powe..."


## 2. Date

In [94]:
# Convert date column to datetime format
data['new_date'] = pd.to_datetime(data['date'], dayfirst=True)
data.head()

,Unnamed: 0,claim,claimant,date,id,label,related_articles,article_array,new_date
0,0,A line from George Orwell's novel 1984 predict...,NaN,17/07/2017,0,0,"[122094, 122580, 130685, 134765]","[122094, 122580, 130685, 134765]",2017-07-17
1,1,Maine legislature candidate Leslie Gibson insu...,NaN,17/03/2018,1,2,"[106868, 127320, 128060]","[106868, 127320, 128060]",2018-03-17
2,2,A 17-year-old girl named Alyssa Carson is bein...,NaN,18/07/2018,4,1,"[132130, 132132, 149722]","[132130, 132132, 149722]",2018-07-18
3,3,In 1988 author Roald Dahl penned an open lette...,NaN,04/02/2019,5,2,"[123254, 123418, 127464]","[123254, 123418, 127464]",2019-02-04
4,4,"When it comes to fighting terrorism, ""Another ...",Hillary Clinton,22/03/2016,6,2,"[41099, 89899, 72543, 82644, 95344, 88361]","[41099, 89899, 72543, 82644, 95344, 88361]",2016-03-22


In [95]:
# create new feature with consecutive days since January 1st, 1986
data['start_date'] = pd.to_datetime('1986-01-01', format='%Y-%m-%d')
data['cont_days'] = (data['new_date'] - data['start_date']).dt.days
data = data.drop(['start_date'], axis=1)

In [96]:
# Convert Year and Month features in to int (instead of str before), can be kept as int since it is ordinal

#Year
data['Year'] = data['new_date'].apply(lambda x: "%d" % (x.year))
data['Year'] = data['Year'].astype(int)
# Month
data['Month'] = data['new_date'].apply(lambda x: "%d" % (x.month))
data['Month'] = data['Month'].astype(int)

In [97]:
# 4 columns at the end show the new_date (which is the date in a date format), the continuous days, 
# the year and month
data.head()

,Unnamed: 0,claim,claimant,date,id,label,related_articles,article_array,new_date,cont_days,Year,Month
0,0,A line from George Orwell's novel 1984 predict...,NaN,17/07/2017,0,0,"[122094, 122580, 130685, 134765]","[122094, 122580, 130685, 134765]",2017-07-17,11520,2017,7
1,1,Maine legislature candidate Leslie Gibson insu...,NaN,17/03/2018,1,2,"[106868, 127320, 128060]","[106868, 127320, 128060]",2018-03-17,11763,2018,3
2,2,A 17-year-old girl named Alyssa Carson is bein...,NaN,18/07/2018,4,1,"[132130, 132132, 149722]","[132130, 132132, 149722]",2018-07-18,11886,2018,7
3,3,In 1988 author Roald Dahl penned an open lette...,NaN,04/02/2019,5,2,"[123254, 123418, 127464]","[123254, 123418, 127464]",2019-02-04,12087,2019,2
4,4,"When it comes to fighting terrorism, ""Another ...",Hillary Clinton,22/03/2016,6,2,"[41099, 89899, 72543, 82644, 95344, 88361]","[41099, 89899, 72543, 82644, 95344, 88361]",2016-03-22,11038,2016,3


## 3. Claimant 

In [98]:
# fill missing claimants with "unknown"
data['claimant'] = data['claimant'].fillna('Unknown')

In [99]:
# Group together all counts less than 100 in to Others
claimant_count = data['claimant'].value_counts()
value_mask = data.claimant.isin(claimant_count.index[claimant_count < 100]) 
data.loc[value_mask,'claimant'] = "Other"

In [100]:
data.head()

,Unnamed: 0,claim,claimant,date,id,label,related_articles,article_array,new_date,cont_days,Year,Month
0,0,A line from George Orwell's novel 1984 predict...,Unknown,17/07/2017,0,0,"[122094, 122580, 130685, 134765]","[122094, 122580, 130685, 134765]",2017-07-17,11520,2017,7
1,1,Maine legislature candidate Leslie Gibson insu...,Unknown,17/03/2018,1,2,"[106868, 127320, 128060]","[106868, 127320, 128060]",2018-03-17,11763,2018,3
2,2,A 17-year-old girl named Alyssa Carson is bein...,Unknown,18/07/2018,4,1,"[132130, 132132, 149722]","[132130, 132132, 149722]",2018-07-18,11886,2018,7
3,3,In 1988 author Roald Dahl penned an open lette...,Unknown,04/02/2019,5,2,"[123254, 123418, 127464]","[123254, 123418, 127464]",2019-02-04,12087,2019,2
4,4,"When it comes to fighting terrorism, ""Another ...",Hillary Clinton,22/03/2016,6,2,"[41099, 89899, 72543, 82644, 95344, 88361]","[41099, 89899, 72543, 82644, 95344, 88361]",2016-03-22,11038,2016,3


## 4. Final Frame

There are 2 final dataframes, one for the data (claims, claimant, date, label, related_articles) and another for the related articles

### 4.1 Final Data 

In [101]:
# concatenate all work done above to a single frame
final_data = pd.concat([data.claim, cleaned_claim.claim, claims_.stemmed_claims, claims_.stemmed_stopword_claims, claims_.tokenized_claims, data.claimant, data.new_date, data.cont_days, data.Year, data.Month, cleaned_claim.label, cleaned_claim.article_array], axis=1)
# rename columns for clarity
final_data.columns = ['raw_claim', 'cleaned_claim', 'stemmed_claims', 'stemmed_stopword_claims', 'tokenized_claim', 'claimant', 'date', 'cont_days', 'year', 'month', 'label', 'article_array']

In [102]:
# this is equivalent to the "train.csv" that we were given, but cleaned with a few additional feature
final_data.head()

,raw_claim,cleaned_claim,stemmed_claims,stemmed_stopword_claims,tokenized_claim,claimant,date,cont_days,year,month,label,article_array
0,A line from George Orwell's novel 1984 predict...,a line from george orwells novel 1984 predicts...,a line from georg orwel novel 1984 predict the...,line georg orwel novel 1984 predict power smar...,"[line, georg, orwel, novel, 1984, predict, pow...",Unknown,2017-07-17,11520,2017,7,0,"[122094, 122580, 130685, 134765]"
1,Maine legislature candidate Leslie Gibson insu...,maine legislature candidate leslie gibson insu...,main legislatur candid lesli gibson insult par...,main legislatur candid lesli gibson insult par...,"[main, legislatur, candid, lesli, gibson, insu...",Unknown,2018-03-17,11763,2018,3,2,"[106868, 127320, 128060]"
2,A 17-year-old girl named Alyssa Carson is bein...,a 17yearold girl named alyssa carson is being ...,a 17yearold girl name alyssa carson is be trai...,17yearold girl name alyssa carson train nasa b...,"[17yearold, girl, name, alyssa, carson, train,...",Unknown,2018-07-18,11886,2018,7,1,"[132130, 132132, 149722]"
3,In 1988 author Roald Dahl penned an open lette...,in 1988 author roald dahl penned an open lette...,in 1988 author roald dahl pen an open letter u...,1988 author roald dahl pen open letter urg par...,"[1988, author, roald, dahl, pen, open, letter,...",Unknown,2019-02-04,12087,2019,2,2,"[123254, 123418, 127464]"
4,"When it comes to fighting terrorism, ""Another ...",when it comes to fighting terrorism another th...,when it come to fight terror anoth thing we kn...,come fight terror anoth thing know doe work ba...,"[come, fight, terror, anoth, thing, know, doe,...",Hillary Clinton,2016-03-22,11038,2016,3,2,"[41099, 89899, 72543, 82644, 95344, 88361]"


In [ ]:
# # save to csv
# final_data.to_csv("final_data.csv")

In [103]:
# save to pkl
final_data.to_pickle("./final_data.pkl")

#### 4.1.1 How to work with final_data.csv

In [ ]:
aaron = pd.read_pickle("./final_data.pkl")

In [ ]:
# to go to a specific cleaned claim
aaron.cleaned_claim.loc[0]

In [ ]:
# to go to a specific stemmed_stopword_claims
aaron.stemmed_stopword_claims.loc[0]

In [ ]:
# sample code to read elements from each article_array

# # to iterate the article array
# for i in range(aaron.shape[0]):
#     # i iterates row by row till the end
#     for u in range(len(aaron.article_array[i])):
#         # u holds the index of each element, within each array. Uncomment the following to understand
#         # print(u)
#         art_array = final_data.article_array[i]
#         # print specific elements of each array
#         print(art_array[u])

### 4.2 Related Articles Dataframe 

In [104]:
raw_article_list = articles.Article.tolist()
cleaned_article_list = articles_cleaned.Article.tolist()
final_articles_zipped = list(zip(raw_article_list, cleaned_article_list, stemmed_art, stemmed_sw_art, tokenized_articles, final_article_sentences, final_article_tok_sentences))
final_articles = pd.DataFrame(final_articles_zipped, columns = ['raw_articles', 'cleaned_articles', 'stemmed_articles', 'stemmed_stopwords_articles', 'tokenized_articles', 'cleaned_sentence', 'tokenized_cleaned_sentence'])
final_articles.index = [articles_cleaned.index]

In [105]:
final_articles.head()

,raw_articles,cleaned_articles,stemmed_articles,stemmed_stopwords_articles,tokenized_articles,cleaned_sentence,tokenized_cleaned_sentence
125385,Dr. Ben Carson: Welfare Benefactor?\nAn initia...,dr ben carson welfare benefactor an initially ...,dr ben carson welfar benefactor an initi unlik...,dr ben carson welfar benefactor initi unlik cl...,"[dr, ben, carson, welfar, benefactor, initi, u...","[dr ben carson welfar benefactor, initi unlik ...","[[dr, ben, carson, welfar, benefactor], [initi..."
32238,The World Factbook — Central Intelligence Agen...,the world factbook central intelligence agency...,the world factbook central intellig agenc the ...,world factbook central intellig agenc unit sta...,"[world, factbook, central, intellig, agenc, un...",[world factbook central intellig agenc unit st...,"[[world, factbook, central, intellig, agenc, u..."
16051,"1014 texaseducationagencypftexas\n\nEmails, La...",1014 texaseducationagencypftexas emails lauren...,1014 texaseducationagencypftexa email lauren c...,1014 texaseducationagencypftexa email lauren c...,"[1014, texaseducationagencypftexa, email, laur...",[1014 texaseducationagencypftexa email lauren ...,"[[1014, texaseducationagencypftexa, email, lau..."
118633,Clinton camp delays Weather Channel ad buy aft...,clinton camp delays weather channel ad buy aft...,clinton camp delay weather channel ad buy afte...,clinton camp delay weather channel ad buy back...,"[clinton, camp, delay, weather, channel, ad, b...",[clinton camp delay weather channel ad buy bac...,"[[clinton, camp, delay, weather, channel, ad, ..."
117945,Living with kangaroos\nKangaroos are appealing...,living with kangaroos kangaroos are appealing ...,live with kangaroo kangaroo are appeal wild an...,live kangaroo kangaroo appeal wild power nativ...,"[live, kangaroo, kangaroo, appeal, wild, power...",[live kangaroo kangaroo appeal wild power nati...,"[[live, kangaroo, kangaroo, appeal, wild, powe..."


In [106]:
final_articles.to_pickle('./final_articles.pkl')

#### 4.2.1 How to work with final_articles.csv

In [107]:
# import the csv
aaron = pd.read_pickle('./final_articles.pkl')
aaron.head()

,raw_articles,cleaned_articles,stemmed_articles,stemmed_stopwords_articles,tokenized_articles,cleaned_sentence,tokenized_cleaned_sentence
125385,Dr. Ben Carson: Welfare Benefactor?\nAn initia...,dr ben carson welfare benefactor an initially ...,dr ben carson welfar benefactor an initi unlik...,dr ben carson welfar benefactor initi unlik cl...,"[dr, ben, carson, welfar, benefactor, initi, u...","[dr ben carson welfar benefactor, initi unlik ...","[[dr, ben, carson, welfar, benefactor], [initi..."
32238,The World Factbook — Central Intelligence Agen...,the world factbook central intelligence agency...,the world factbook central intellig agenc the ...,world factbook central intellig agenc unit sta...,"[world, factbook, central, intellig, agenc, un...",[world factbook central intellig agenc unit st...,"[[world, factbook, central, intellig, agenc, u..."
16051,"1014 texaseducationagencypftexas\n\nEmails, La...",1014 texaseducationagencypftexas emails lauren...,1014 texaseducationagencypftexa email lauren c...,1014 texaseducationagencypftexa email lauren c...,"[1014, texaseducationagencypftexa, email, laur...",[1014 texaseducationagencypftexa email lauren ...,"[[1014, texaseducationagencypftexa, email, lau..."
118633,Clinton camp delays Weather Channel ad buy aft...,clinton camp delays weather channel ad buy aft...,clinton camp delay weather channel ad buy afte...,clinton camp delay weather channel ad buy back...,"[clinton, camp, delay, weather, channel, ad, b...",[clinton camp delay weather channel ad buy bac...,"[[clinton, camp, delay, weather, channel, ad, ..."
117945,Living with kangaroos\nKangaroos are appealing...,living with kangaroos kangaroos are appealing ...,live with kangaroo kangaroo are appeal wild an...,live kangaroo kangaroo appeal wild power nativ...,"[live, kangaroo, kangaroo, appeal, wild, power...",[live kangaroo kangaroo appeal wild power nati...,"[[live, kangaroo, kangaroo, appeal, wild, powe..."


In [108]:
# to go to a specific cleaned article
aaron.cleaned_articles.loc['75770'].iloc[0]

'charterschool bill deserves action ohio senate president keith faber who is scouting a run for state auditor or attorney general in two years has a chance to demonstrate an affinity for defending ohioans against misspending and fraud by advancing a charterschool accountability bill ohio senate president keith faber who is scouting a run for state auditor or attorney general in two years has a chance to demonstrate an affinity for defending ohioans against misspending and fraud by advancing a charterschool accountability bill as it stands it appears that fabers senate is playing games with senate bill 298 a measure to ensure that charter schools are actually educating students this crucial reform legislation was oddly assigned to the senate finance committee rather than being sent to the education committee where sen peggy lehner rkettering had made it clear that shed fasttrack the bill senate minority leader joe schiavoni dboardman reasonably suspects his bill is being slotted for ina

In [111]:
# to go to a specific raw article
aaron.tokenized_cleaned_sentence.loc['75770'].iloc[0]

[['charterschool',
  'bill',
  'deserv',
  'action',
  'ohio',
  'senat',
  'presid',
  'keith',
  'faber',
  'scout',
  'run',
  'state',
  'auditor',
  'attorney',
  'gener',
  'two',
  'year',
  'ha',
  'chanc',
  'demonstr',
  'affin',
  'defend',
  'ohioan',
  'misspend',
  'fraud',
  'advanc',
  'charterschool',
  'account',
  'bill'],
 ['ohio',
  'senat',
  'presid',
  'keith',
  'faber',
  'scout',
  'run',
  'state',
  'auditor',
  'attorney',
  'gener',
  'two',
  'year',
  'ha',
  'chanc',
  'demonstr',
  'affin',
  'defend',
  'ohioan',
  'misspend',
  'fraud',
  'advanc',
  'charterschool',
  'account',
  'bill'],
 ['stand',
  'appear',
  'faber',
  'senat',
  'play',
  'game',
  'senat',
  'bill',
  '298',
  'measur',
  'ensur',
  'charter',
  'school',
  'actual',
  'educ',
  'student'],
 ['thi',
  'crucial',
  'reform',
  'legisl',
  'wa',
  'oddli',
  'assign',
  'senat',
  'financ',
  'committe',
  'rather',
  'sent',
  'educ',
  'committe',
  'sen',
  'peggi',
  'leh